In [1]:
import pandas as pd
import numpy as np
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
#from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OrdinalEncoder


In [2]:
import seaborn as sns

In [3]:
impute_data = sns.load_dataset('titanic')
pd.options.display.max_columns = None
impute_data.head()

survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True

In [4]:
impute_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


In [5]:
impute_data['deck1'] = impute_data['deck'].astype(object)
impute_data['class1'] = impute_data['class'].astype(object)

In [6]:
impute_data = impute_data.drop(['deck','class'], axis = 1)

In [7]:
impute_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   who          891 non-null    object 
 9   adult_male   891 non-null    bool   
 10  embark_town  889 non-null    object 
 11  alive        891 non-null    object 
 12  alone        891 non-null    bool   
 13  deck1        203 non-null    object 
 14  class1       891 non-null    object 
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB


In [8]:
#instantiate both packages to use
encoder = OrdinalEncoder()
imputer = IterativeImputer(ExtraTreesRegressor())
# create a list of categorical columns to iterate over
cat_cols = ['embarked','class1','deck1','who','embark_town','sex','adult_male','alive','alone']

def encode(data):
    '''function to encode non-null data and replace it in the original data'''
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data

#create a for loop to iterate through each column in the data
for columns in cat_cols:
    encode(impute_data[columns])

C:\Users\PC-1\AppData\Local\Programs\Python\Python38\Lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\PC-1\AppData\Local\Programs\Python\Python38\Lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\PC-1\AppData\Local\Programs\Python\Python38\Lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyd

In [9]:
impute_data

survived  pclass  sex   age  sibsp  parch     fare embarked  who  \
0           0       3  1.0  22.0      1      0   7.2500        2  1.0   
1           1       1  0.0  38.0      1      0  71.2833        0  2.0   
2           1       3  0.0  26.0      0      0   7.9250        2  2.0   
3           1       1  0.0  35.0      1      0  53.1000        2  2.0   
4           0       3  1.0  35.0      0      0   8.0500        2  1.0   
..        ...     ...  ...   ...    ...    ...      ...      ...  ...   
886         0       2  1.0  27.0      0      0  13.0000        2  1.0   
887         1       1  0.0  19.0      0      0  30.0000        2  2.0   
888         0       3  0.0   NaN      1      2  23.4500        2  2.0   
889         1       1  1.0  26.0      0      0  30.0000        0  1.0   
890         0       3  1.0  32.0      0      0   7.7500        1  1.0   

     adult_male embark_town  alive  alone deck1  class1  
0           1.0           2    0.0    0.0   NaN     2.0  
1           0.0           0    1.0    0.0     2     0.0  
2           0.0           2    1.0    1.0   NaN     2.0  
3           0.0           2    1.0    0.0     2     0.0  
4           1.0           2    0.0    1.0   NaN     2.0  
..          ...         ...    ...    ...   ...     ...  
886         1.0           2    0.0    1.0   NaN     1.0  
887         0.0           2    1.0    1.0     1     0.0  
888         0.0           2    0.0    0.0   NaN     2.0  
889         1.0           0    1.0    1.0     2     0.0  
890         1.0           1    0.0    1.0   NaN     2.0  

[891 rows x 15 columns]

In [10]:
# impute data and convert 
encode_data = pd.DataFrame(np.round(imputer.fit_transform(impute_data)),columns = impute_data.columns)

C:\Users\PC-1\AppData\Local\Programs\Python\Python38\Lib\site-packages\sklearn\impute\_iterative.py:637: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


In [11]:
encode_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    float64
 1   pclass       891 non-null    float64
 2   sex          891 non-null    float64
 3   age          891 non-null    float64
 4   sibsp        891 non-null    float64
 5   parch        891 non-null    float64
 6   fare         891 non-null    float64
 7   embarked     891 non-null    float64
 8   who          891 non-null    float64
 9   adult_male   891 non-null    float64
 10  embark_town  891 non-null    float64
 11  alive        891 non-null    float64
 12  alone        891 non-null    float64
 13  deck1        891 non-null    float64
 14  class1       891 non-null    float64
dtypes: float64(15)
memory usage: 104.5 KB


In [12]:
target = encode_data.fare
features = encode_data.drop('fare', axis=1)

In [13]:
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer, PowerTransformer

In [14]:
ss = StandardScaler()
rs = RobustScaler()
qt = QuantileTransformer(output_distribution='normal',n_quantiles=891)
yj = PowerTransformer(method = 'yeo-johnson')
bc = PowerTransformer(method = 'box-cox')

In [15]:
ss_data = ss.fit_transform(features)
rs_data = rs.fit_transform(features)
qt_data = qt.fit_transform(features)
yj_data = yj.fit_transform(features)
#bc_data = pd.DataFrame(bc.fit_transform(encode_data), columns = encode_data.columns)

In [16]:
from sklearn.linear_model import LinearRegression

In [17]:
lr = LinearRegression()

In [18]:
lr.fit(ss_data, target)
lr.score(ss_data, target)

0.42438303340085604

In [19]:
lr.fit(rs_data, target)
lr.score(rs_data, target)

0.42466241821408945

In [20]:
lr.fit(qt_data, target)
lr.score(qt_data, target)

0.3869558885820199

In [21]:
lr.fit(features, target)
lr.score(features, target)

0.42466241821408945

In [22]:
lr.fit(yj_data, target)
lr.score(yj_data, target)

0.4431580972443995